In [ ]:
#Task 1 Establishing a connection pool
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

dbconfig = {
    "database":"little_lemon_db",
    "user":"root",
    "password":"Youseethis123"
}

try: 
    pool = MySQLConnectionPool(pool_name = "pool_b", pool_size = 2, **dbconfig)
    print("The connection to pool is created with a name: ", pool.pool_name)
    print("The pool size is: ", pool.pool_size)

except Error as er:
    print("Error code: ", er.errno)
    print("Error message: ", er.msg)

In [ ]:
#Task 2 Booking Dinner slots, I decided to just do the queries first
booking1 = """INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
               VALUES
               (8, 'Anees', 'Java', '18:00', 6);"""
booking2 = """INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
               VALUES
               (5, 'Bald', 'Vin', '19:00', 6);"""
booking3 = """INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
               VALUES
               (12, 'Jay', 'Kon', '19:30', 6);"""

In [ ]:
connection1 = pool.get_connection()
cursor1 = connection1.cursor()

cursor1.execute(booking1)
connection1.commit()
print("Successfully added.")

In [ ]:
connection2 = pool.get_connection()
cursor2 = connection2.cursor()

cursor2.execute(booking2)
connection2.commit()
print("Successfully added.")

In [ ]:
import mysql.connector as connector

In [ ]:
#Need to create new connection
try:
    connection3 = pool.get_connection()
    print("You have been connected.")
except:
    print("No more connections, please wait while one is created...")
    connection = connector.connect(user = "root", password = "Youseethis123")
    pool.add_connection(cnx=connection)
    connection3 = pool.get_connection()
    print("New connection added.")

cursor3 = connection3.cursor()
cursor3.execute(booking3)
connection3.commit()
print("Successfully added.")

In [ ]:
#Returning connections to pool with error message for the extra *new* connection
for connection in [connection1, connection2, connection3]:
    try:
        connection.close()
        print("Returned to pool")
    except Error as err:
        print("Extra connection cannot be added back to pool. ")
        print("Error code: ", err.errno)
        print("Error message: ", err.msg)

In [ ]:

#Report 1 (Task 3)
report_1A = """SELECT Name, EmployeeID FROM Employees WHERE Role = 'Manager'; """
report_1B = """SELECT Name, Role FROM Employees ORDER BY Annual_Salary DESC LIMIT 1; """
report_1C = """SELECT COUNT(BookingID) AS TotalBookings FROM Bookings WHERE BookingSlot BETWEEN '18:00' AND '20:00'; """
report_1D = """SELECT CONCAT(GuestFirstName,' ', GuestLastName) AS FullName, BookingID 
                FROM Bookings LEFT JOIN Employees ON Bookings.EmployeeID = Employees.EmployeeID 
                WHERE Employees.Role = 'Receptionist'; """

cursor = connection.cursor()
cursor.execute(report_1A)
results1 = cursor.fetchall()
cols = cursor.column_names
print(cols)
for results in results1:
    print(results, "\n")

cursor.execute(report_1B)
results1 = cursor.fetchall()
cols = cursor.column_names
print(cols)
for results in results1:
    print(results, "\n")

cursor.execute(report_1C)
results1 = cursor.fetchall()
cols = cursor.column_names
print(cols)
for results in results1:
    print(results, "\n")

cursor.execute(report_1D)
results1 = cursor.fetchall()
cols = cursor.column_names
print(cols)
for results in results1:
    print(results)

In [36]:
#Task 4 BasicSalesReport
#cursor.execute("""DROP PROCEDURE IF EXISTS BasicSalesReport""")

sql_procedure_BasicSalesReport = """
CREATE PROCEDURE BasicSalesReport()
BEGIN

SELECT SUM(BillAmount) AS Total_Sales, ROUND(AVG(BillAmount), 2) AS Average_Bill_Amount,
MIN(BillAmount) AS Minimum_Bill_Paid, MAX(BillAmount) AS Maximum_Bill_Paid
FROM Orders;

END
"""

In [37]:
cursor.execute(sql_procedure_BasicSalesReport)

In [38]:
cursor.callproc("BasicSalesReport")

()

In [39]:
results = next(cursor.stored_results())
dataset = results.fetchall()

for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]
print(columns)

for data in dataset:
    print(data)

['Total_Sales', 'Average_Bill_Amount', 'Minimum_Bill_Paid', 'Maximum_Bill_Paid']
(Decimal('243'), Decimal('48.60'), 37, 86)


In [40]:
#Task 5 Upcoming Bookings
#Get connection and create buffered cursor
chef_connection = pool.get_connection()
cursor = chef_connection.cursor(buffered = True)

In [46]:
#SQL statement to retrieve upcoming 3 bookings
chef_query = """
SELECT BookingSlot, CONCAT(GuestFirstName, ' ', GuestLastName) AS Guest_name, Name, Role
FROM Bookings LEFT JOIN Employees ON Bookings.EmployeeID = Employees.EmployeeID
ORDER BY BookingSlot ASC LIMIT 3;
"""

cursor.execute(chef_query)
results = cursor.fetchall()
for results in results:
    print("Booking Slot: {} \nGuest Name: {} \nAssigned to: {} [{}]".format(results[0], results[1], results[2], results[3]))
    print("\n")

Booking Slot: 15:00:00 
Guest Name: Vanessa McCarthy 
Assigned to: Giorgos Dioudis [Head Chef]


Booking Slot: 17:30:00 
Guest Name: Marcos Romero 
Assigned to: Fatma Kaya [Assistant Chef]


Booking Slot: 18:00:00 
Guest Name: Anees Java 
Assigned to: John Millar [Receptionist]




In [47]:
#Returning connection to the pool
try:
    chef_connection.close()
    print("Returned to pool")
except Error as err:
    print("Error code: ", err.errno)
    print("Error message: ", err.msg)

Returned to pool
